In [1]:
import Control.Lens
import Control.Applicative 
import Data.Char
import qualified Data.Map as M 
import qualified Data.Set as S 
import qualified Data.Text as T

In [2]:
:extension TemplateHaskell
:extension FlexibleInstances
:extension FlexibleContexts
:extension RankNTypes
:extension ScopedTypeVariables
:extension TypeApplications
:extension TypeFamilies
:extension InstanceSigs

In [3]:
import Data.Ord (comparing)

checkPalindrome :: String -> Bool
checkPalindrome s = reverse s == s

input1 = ["umbrella", "olives", "racecar", "hammer"]
findOf folded checkPalindrome input1

input2 = (2, 4, 6)
allOf each even input2

input3 = [(2, "I'll"), (3, "Be"), (1, "Back")]
maximumByOf folded (comparing fst) input3


countVowels :: String -> Int
countVowels = length . filter (\c -> elemOf folded c "aeiou")

input4 = "Do or do not, there is no try."
maximumByOf worded (comparing countVowels) input4

input5 = ["a", "b", "c"]
foldrOf folded (flip (++)) mempty input5

input6 = [(12, 45, 66), (91, 123, 87)]
toListOf (folded . _2 . to (reverse . show) . folded) input6

input7 = [(1, "a"), (2, "b"), (3, "c"), (4, "d")]
foldMapOf folded (\(a , r) -> [r | even a]) input7


Just "racecar"

True

Just (3,"Be")

Just "there"

"cba"

"54321"

["b","d"]

In [4]:
-- higher order folds

"Here's looking at you, kid" ^.. dropping 7 folded

["My Precious", "Hakuna Matata", "No problemo"] ^.. folded . taking 1 worded

["My Precious", "Hakuna Matata", "No problemo"] ^.. taking 1 (folded . worded)

["My Precious", "Hakuna Matata", "No problemo"] ^.. folded . taking 1 worded . folded

sumOf (taking 2 each) (10, 50, 100)

("stressed", "guns", "evil") ^.. backwards each

("stressed", "guns", "evil") ^.. backwards each . to reverse
 
"blink182 k9 blazeit420" ^.. worded . droppingWhile isAlpha folded

sample :: [Int]
sample = [-10, -5, 4, 3, 8, 6, -2, 3, -5, -7]

lengthOf (takingWhile (<= 0) folded) sample

maximumOf (taking 4 folded) sample

(lengthOf . backwards) (takingWhile (<= 0) folded) sample

sample ^.. droppingWhile (<= 0) folded ^.. takingWhile (> 0) folded

(sample ^.. droppingWhile (<= 0) folded) ^.. backwards (droppingWhile (<= 0) (backwards folded))

trimmingWhile :: (a -> Bool) -> Fold s a -> Fold s a
trimmingWhile f s = backwards $ droppingWhile f $ backwards $ droppingWhile f s

sample ^.. trimmingWhile (<= 0) folded


"looking at you, kid"

["My","Hakuna","No"]

["My"]

"MyHakunaNo"

60

...

Just 4

2

[4,3,8,6]

[4,3,8,6,-2,3]

[4,3,8,6,-2,3]

In [44]:
-- simple traversals
("Jurassic", "Park") & both .~ "N/A"
("Jurassic", "Park") & both . traversed .~ 'x'
("Malcolm", ["Kaylee", "Inara", "Jayne"]) & beside id traversed %~ take 3
("Malcolm", ["Kaylee", "Inara", "Jayne"]) & _2 . element 1 .~ "River"
["Die Another Day", "Live and Let Die", "You Only Live Twice"] & traversed . elementOf worded 1 . traversed .~ 'x'
((1, 2), (3, 4)) & both . both +~ 1
(1, (2, [3, 4])) & beside id (beside id traversed) +~ 1

import Data.Char (toUpper)

((True, "Strawberries"), (False, "Blueberries"), (True, "Blackberries")) 
    & each . filtered ( (== True) . fst) . _2 . taking 4 traversed %~ toUpper
    
((True, "Strawberries"), (False, "Blueberries"), (True, "Blackberries")) & each %~ snd

("N/A","N/A")

("xxxxxxxx","xxxx")

("Mal",["Kay","Ina","Jay"])

("Malcolm",["Kaylee","River","Jayne"])

["Die xxxxxxx Day","Live xxx Let Die","You xxxx Live Twice"]

((2,3),(4,5))

(2,(3,[4,5]))

((True,"STRAwberries"),(False,"Blueberries"),(True,"BLACkberries"))

("Strawberries","Blueberries","Blackberries")

In [77]:
-- traversal actions
sequenceAOf (traversed . _1) [("ab", 1), ("cd", 2)]

sequenceAOf traversed [ZipList [1, 2], ZipList [3, 4]]
sequenceAOf (traversed . _2) [('a', ZipList [1,2]), ('b', ZipList [3,4])]

import Control.Monad.State

let result = traverseOf (beside traversed both)
                        (\n -> modify (+n) >> get) 
                        ([1, 1, 1], (1, 1))
                        
runState result 0


-- data User = User { 
--     _name :: String
--     , _age :: Int
--     } deriving Show 
-- makeLenses ''User

-- data Account = Account { 
--     _id :: String
--     , _user :: User
--     } deriving Show
-- makeLenses ''Account

-- validateAge :: Account -> Either String Account 
-- validateAge = (user . age) %%~ (\u -> if (u > 0 && u < 150) then Right u else Left "Error" )
-- validateAge = _




[[('a',1),('c',2)],[('a',1),('d',2)],[('b',1),('c',2)],[('b',1),('d',2)]]

ZipList {getZipList = [[1,3],[2,4]]}

ZipList {getZipList = [[('a',1),('b',3)],[('a',2),('b',4)]]}

(([1,2,3],(4,5)),5)

In [48]:
ZipList [1,2,3]

ZipList {getZipList = [1,2,3]}